In [6]:
import numpy as np
import pandas as pd
import requests
import datetime
import matplotlib.pyplot as plt

# Temperature data


In [3]:
api_key = 'WBmYeqGTZMphjzecIADAthdwDyQUcnbL'
# url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/datasets'
url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND"
params = {
    "id": "GHCND",
    "name": "Daily Summaries",
    "datacoverage": 1,
    "startdate": "2018-01-01",
    "enddate": "2018-01-03"
}
headers = {
    'token': api_key
}
response = requests.get(url, headers=headers, params=params)
data = response.json()
df = pd.json_normalize(data, 'results')
print(df.head())

                  date datatype            station attributes  value
0  2018-01-01T00:00:00     TAVG  GHCND:AE000041196      H,,S,    186
1  2018-01-01T00:00:00     TMAX  GHCND:AE000041196       ,,S,    259
2  2018-01-01T00:00:00     TMIN  GHCND:AE000041196       ,,S,    112
3  2018-01-01T00:00:00     PRCP  GHCND:AEM00041194       ,,S,      0
4  2018-01-01T00:00:00     TAVG  GHCND:AEM00041194      H,,S,    209


In [ ]:
def get_noaa_data(api_key, station_id, start_date, end_date):
    base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    dataset = "GHCND"
    params = {
        "datasetid": dataset,
        "stationid": station_id,
        "startdate": start_date,
        "enddate": end_date,
        "units": "metric",
        "limit": 1000
    }
    headers = {
        "token": api_key
    }
    response = requests.get(base_url, headers=headers, params=params)
    return response.json()

In [ ]:
station_id = {
    "KARB" : ["GHCND:USC00200228"], # Ann Arbor Airport 
    "PANC" : "GHCND:USW00026451",
    "KBOI" : "GHCND:USW00024131",
    "KORD" : "GHCND:USW00094846",
    "KDEN" : "GHCND:USW00023062",
    "KDTW" : "GHCND:USW00014838",
    "PHNL" : "GHCND:USW00022521",


}

In [ ]:
new_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/stations"

# Jonathan's data and code

In [9]:
endpoint = "https://aviationweather.gov/api/data/metar"
stations = ["PANC"]  # and other stations too
params = {"ids": stations, "format": "json", "hours": 48}
response = requests.get(endpoint, params=params)
df = pd.DataFrame.from_records(response.json())

In [8]:
df["reportTime"] = pd.to_datetime(df["reportTime"])
df = df.loc[
    (df["reportTime"].dt.date == datetime.date(2023, 11, 19))  # or whatever the date is
    & (df["metarType"] == "METAR")
][["temp", "precip", "snow"]].agg(["mean", "max", "min", "count"])

y_temp = df["temp"][:3]
y_snow = df["snow"]["max"] > 0
y_rain = df["precip"]["max"] > 0

# Sahana time